# Install

In [1]:
!pip install spotipy

# Imports

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
import pandas as pd
from __future__ import print_function
import json
import time
import sys

# Credentials from Spotify's API

In [9]:
client_id= CLIENT_ID
client_secret= CLIENT_SECRET
pl_id = 'spotify:playlist:5ABHKGoOzxkaa28ttQV9sE' # to be changed
#top50_tracks_2022: 37i9dQZF1DX18jTM2l2fJY
#top50_tracks_2021:
#top50_tracks_2020: 37i9dQZF1DX7Jl5KP2eZaS
#top100_tracks_2019: 37i9dQZF1DWVRSukIED0e9
#top100_tracks_2018: 37i9dQZF1DXe2bobNYDtW8
#top50_tracks_weekly: 37i9dQZEVXbNG2KDcFcKOF
#top50_tracks_daily: 37i9dQZEVXbMDoHDwVN2tF


# Main Features

## Getting main features from playlist

In [5]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id= client_id, client_secret= client_secret))

In [6]:
track_id= [sp.playlist(pl_id)['tracks']['items'][i]['track']['id'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [ ]:
name= [sp.playlist(pl_id)['tracks']['items'][i]['track']['name'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [ ]:
popularity= [sp.playlist(pl_id)['tracks']['items'][i]['track']['popularity'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [ ]:
duration_ms= [sp.playlist(pl_id)['tracks']['items'][i]['track']['duration_ms'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [ ]:
explicit= [sp.playlist(pl_id)['tracks']['items'][i]['track']['explicit'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [ ]:
artist_name= [sp.playlist(pl_id)['tracks']['items'][i]['track']['artists'][0]['name'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [ ]:
artist_id= [sp.playlist(pl_id)['tracks']['items'][i]['track']['artists'][0]['id'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [ ]:
release_date= [sp.playlist(pl_id)['tracks']['items'][i]['track']['album']['release_date'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

## Features into dataframe

In [ ]:
main_df= pd.DataFrame({'track_id': track_id,
                  'track_name': name,
                  'popularity': popularity,
                  'duration_ms': duration_ms,
                  'explicit': explicit,
                  'artist_name': artist_name,
                  'artist_id': artist_id,
                  'release_date': release_date},
                  columns=['track_id','track_name','popularity','duration_ms','explicit','artist_name','artist_id','release_date'])
main_df

# Audio Features

## Getting audio features from playlist

In [ ]:
tracks=[]

for track_id in df.loc[:, 'track_id']:
    track=f'spotify:track:{track_id}'
    audio_features= sp.audio_features(track)
    tracks.append(audio_features[0])

## Audio features into a clean dataframe

In [ ]:
audio_df= pd.DataFrame(tracks)

In [ ]:
audio_df= audio_df.drop(['type','uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis=1)

In [ ]:
audio_df= audio_df.rename(columns={"id": "track_id"})
audio_df

# Combining the 2 dataframes

In [ ]:
spotify_df= main_df.merge(audio_df, on='track_id')
spotify_df

# Adding preview_url

In [ ]:
previews=[]

for track_id in spotify_df.loc[:, 'track_id']:
    urn=f'spotify:track:{track_id}'
    if sp.track(urn)['preview_url']:
        preview_url= sp.track(urn)['preview_url']
        previews.append(preview_url)
    else:
        previews.append('### no audio preview available ###')
previews

In [ ]:
preview_df= pd.DataFrame(previews, columns=['preview_url'])
preview_df

# Adding cover_url

In [ ]:
covers=[]

for track_id in spotify_df.loc[:, 'track_id']:
    urn=f'spotify:track:{track_id}'
    if sp.track(urn)['album']['images'][0]['url']:
        cover_url= sp.track(urn)['album']['images'][0]['url']
        covers.append(cover_url)
    else:
        covers.append('### no cover available ###')
covers

In [ ]:
covers_df= pd.DataFrame(covers, columns=['cover_url'])
covers_df

# Combining all dataframe in one

In [ ]:
playlist_df = pd.concat([spotify_df, preview_df, covers_df ], axis=1)
playlist_df

# Export as csv

In [ ]:
playlist_df.to_csv('playlist_df.csv')